### Проект. Анна Запорощенко, Софья Генералова

### Предобработка текстов

Вам нужно собрать свой корпус текстов (100 штук не менее 100 слов каждый / 10 000 словоупотреблений). Корпус состоит из текстов одного жанра или тематики (не отрывки одного текста).

Можно делать это вручную или скачивая html-страницы средствами Питона (например, библиотека requests). Затем нужно удалить нетекстовые элементы, разделить корпус на предложения. Не забудьте, что для каждого предложения нужно помнить, из какого источника оно берется.

In [1]:
!pip install fake_useragent

In [18]:
import re

In [19]:
import urllib.request
from fake_useragent import UserAgent
import random

In [3]:
# функция возвращает html код страницы
def get_html_content(url):
    user_agent = UserAgent().chrome
    req = urllib.request.Request(url, headers={'User-Agent':user_agent})
    with urllib.request.urlopen(req) as response:
        html_content = response.read().decode('utf-8')
    return html_content

In [4]:
# функция возвращает ссылки на новости, собранные на странице
def get_links(html_content, pattern, links):
    l = re.findall(pattern, html_content)
    links.extend(l)
    return links

In [5]:
# инициализируем список для ссылок
# генерируем номера страниц со ссылками на новости (первая без номера, ее оставим) и достаем ссылки из них
links = []
pages = [t for t in range(2,7)]
pattern = '<a href=\'(.+?)\'><h3 class="news-big"'
for n in pages:
    url = 'https://liveberlin.ru/news/'
    url = url + 'page/' + str(n)
    links = get_links(get_html_content(url), pattern, links)

In [6]:
# функция убирает нерасшифровавшиеся знаки, рекламу, теги в строке
def clear(line):
    line = line.replace('&nbsp;', ' ')
    line = line.replace('&#8217;', "'")
    line = line.replace('\xa0', ' ')
    line = re.split('<.+?>', line)
    line = ('').join(line)
    line = line.replace(' Реклама в «Живом Берлине» | liveberlin.ad@gmail.com', '')
    line = re.sub('\s+', ' ',line) # убирает повторяющиеся пробельные символы
    return line

In [7]:
# функция достает из html кода новостной страницы текст новости и заголовок
def get_info(html_content):
    text = clear((' ').join(re.findall('<p>.+?</p>', html_content)[:-2]))
    title = clear(('').join(re.findall("<meta property=\'og:type\' content=\'article\' />\n    <meta property=\'og:title\' content=\'(.+?)' />", html_content)))
    return text, title

In [8]:
texts = []
titles = []

In [9]:
for link in links:
    html_content = get_html_content(link)
    text, title = get_info(html_content)
    texts.append(text)
    titles.append(title + '\n' + link)

### Морфологический анализ текстов

Здесь можно использовать любой инструмент / библиотеку, которая позволяет проводить морфологический анализ.
Подумайте, нужно ли вам разрешение морфологической неоднозначности. Хорошо бы сказать об этом в презентации и обосновать выбранное решение.
Не забудьте, что для запросов тоже понадобится морфологический анализ.

Natasha делит по предложениям.

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

In [4]:
segmenter = Segmenter()

emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)

morph_vocab = MorphVocab()

функция sent_a получает на вход текст и информацию о нем, делит на предложения, размечает части речи, и отправляет каждое предложение в функцию sent_n_gramm

In [12]:
def sent_a(doc, text_meta, start=0):
    next_start = 0
    for sent_id, sent in enumerate(doc.sents, start):
        next_start = sent_id
        sent_text = sent.text
#         print(sent_text)
        sentence = []
        for tok in sent.tokens:
            pos = tok.pos
            if pos != 'PUNCT':
                tok.lemmatize(morph_vocab)
                lem = tok.lemma
                tex = tok.text
                sentence.append(tex + '<pos:' + pos + '>' + '<lem:' + lem + '>')
        # ДЕЛИТ НА ЭНГРАММЫ
        sent_n_gramm(sentence, sent_text, sent_id, text_meta)
        
    return next_start+1

In [13]:
big_dict = {}

функция sent_n_gramm делит полученное предложение на энграммы, записывает в словарь

In [14]:
def sent_n_gramm(mas_for_sent, sent_text, sent_id, text_meta):
    rr = []
    ln = len(mas_for_sent)
    if ln < 3 and ln > 0:
        rr.append(' '.join(mas_for_sent))
    else:
        for i in range(ln):
            if i < ln-2:
                res = mas_for_sent[i] + ' ' + mas_for_sent[i+1] + ' ' + mas_for_sent[i+2]
                rr.append(res)
                
    for n_gr in rr:
        ws = n_gr.split('> ')
        ngrm = [] # list from which i will concatenate strs
        for w in ws: # для кажд части энграммы
            w_text = re.search('([^<>]+)<', w).group(1)
            ngrm.append(w_text)
    
        ngrm_s = ' '.join(ngrm)
        if ngrm_s not in big_dict:
            big_dict[ngrm_s] = {'sent_id': [], 'text_meta':[], 'sent_text':[], 'tag_ngr':[]}
            
        if sent_id not in big_dict[ngrm_s]['sent_id'] or n_gr not in big_dict[ngrm_s]['tag_ngr']:
            big_dict[ngrm_s]['sent_id'].append(sent_id)
            big_dict[ngrm_s]['text_meta'].append(text_meta)
            big_dict[ngrm_s]['sent_text'].append(sent_text)
            big_dict[ngrm_s]['tag_ngr'].append(n_gr)

In [15]:
next_start = 0
for text, text_meta in zip(texts, titles):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    next_start = sent_a(doc, text_meta, next_start) # след начальный айди предложения

In [16]:
len(big_dict)

13782

13782 3gramm

In [5]:
import sqlite3

In [18]:
#подключаемся к базе данных
conn = sqlite3.connect('project_db.db')

In [19]:
cur = conn.cursor()

Таблица для предложений

In [20]:
cur.execute("CREATE TABLE IF NOT EXISTS 'Sentences'('sent_id' integer PRIMARY KEY AUTOINCREMENT, 'title_link', 'sent_orig')")

Таблица n-грамм

In [21]:
cur.execute("CREATE TABLE IF NOT EXISTS 'Corpus'('ngr_id' integer PRIMARY KEY AUTOINCREMENT, '3-gram0_orig', '3-gram1_orig', '3-gram2_orig', '3-gram0_lem', '3-gram1_lem', '3-gram2_lem', '3-gram0_pos', '3-gram1_pos', '3-gram2_pos', 'sent' integer, FOREIGN KEY('sent') REFERENCES 'Sentences'('sent_id'))")

In [22]:
conn.commit()
conn.close()

Из словаря big_dict все записывается в базу данных

In [23]:
conn = sqlite3.connect('project_db.db')
c = conn.cursor()

In [24]:
countt = 0 # счетчик для энграмм
for gr in big_dict:
    for i in range(len(big_dict[gr]['sent_id'])):
        sent_id = big_dict[gr]['sent_id'][i]
            
        title_link = big_dict[gr]['text_meta'][i]
        sent_orig = big_dict[gr]['sent_text'][i]
        ngr_id = big_dict[gr]['tag_ngr'][i]
        
        ws = ngr_id.split('> ')
        
        # могут быть не только триграммы, но и биграммы, униграммы (теоретически - если предложение состояло из 1-2 слов)
        
        n_gram1_orig, n_gram1_lem, n_gram1_pos = '', '', ''
        n_gram2_orig, n_gram2_lem, n_gram2_pos = '', '', ''
        
        for j in range(len(ws)):
            w_text = re.search('([^<>]+)<', ws[j]).group(1)
            pos = re.search('<pos:([^>]+)>', ws[j]).group(1)
            lem = re.search('<lem:([^>]+)>?', ws[j]).group(1)
            if j == 0:
                n_gram0_orig = w_text
                n_gram0_lem = lem
                n_gram0_pos = pos
            elif j == 1:
                n_gram1_orig = w_text
                n_gram1_lem = lem
                n_gram1_pos = pos
            else:
                n_gram2_orig = w_text
                n_gram2_lem = lem
                n_gram2_pos = pos
        
        c.execute('INSERT OR IGNORE INTO "Sentences" VALUES (?, ?, ?)', (sent_id, title_link, sent_orig)) 
        # если попробует добавить с повт праймари ки, то проигнорит
        c.execute('INSERT INTO "Corpus" VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (countt, n_gram0_orig, n_gram1_orig, n_gram2_orig, n_gram0_lem, n_gram1_lem, n_gram2_lem, n_gram0_pos, n_gram1_pos, n_gram2_pos, sent_id))
        conn.commit()
        countt += 1
        

In [25]:
conn.commit()
conn.close()

### Функция поиска

поиск - нужно найти все предложения, где слово встречается в любой форме
“поиска” - нужно найти предложения только с этой формой
знать+NOUN - нужно найти все предложения, где встречается существительное “знать”
NOUN - найти все предложения с существительными

В питоне это, скорее всего, будет выглядеть так:

* search(‘поиск’)
* search(‘“поиск”’) - дублируем кавычки
* search(‘знать+NOUN’)
* search(‘NOUN VERB ADV’)
* search(‘ADJ дом’)

Запрос состоит из последовательных слов/POS-тегов (n-грамма,  максимум 3-грамма), к каждому применяются правила выше.

Запросы могут состоять из 

1) леммы/словоформы (тогда нужно найти вхождения во всех формах), 

2) словоформы в двойных кавычках (тогда нужно найти только заданную форму), 

3) леммы и POS-тега (тогда нужно найти все формы, отмеченные данным тегом),

4) POS-тега/ов (тогда нужно найти все слова/n-граммы, соответствующие тегу/набору тегов).

Выдача должна состоять из предложений с мета-информацией (например, URL страницы, откуда взят текст, или название текста + название источника - например, “Война и мир” - lib.ru)


In [6]:
list_of_pos = set(['ADJ',
'ADV',
'DET',
'AUX',
'CCONJ',
'SCONJ',
'VERB',
'INTJ',
'NOUN',
'PROPN',
'PRON',
'NUM',
'SYM',
'PART',
'ADP',
'X']) # тег 'PUNCT' будет считаться словом

функция generate_request создает подзапросы напр. "Corpus.'3-gram@_lem' = 'кошка'"

In [7]:
def generate_request(input_s):
    res_words = []
    
    words = input_s.split(' ') # кошек "нет" NOUN
    for w_index, word in enumerate(words):
        if word[0] == '"':
            if word[-1] != '"': # если кавычка не закрыта
                res_words = []
                print('Попробуйте закрыть двойные кавычки')
                return res_words
            
            #поиск по форме слова
            line = "Corpus.'3-gram@_orig' = " + "'" + word.strip('"') + "'"
        elif word in list_of_pos:
            #поиск по POS
            line = "Corpus.'3-gram@_pos' = " + "'" + word + "'"
        elif '+' in word:
            if word[0] == '+' or word[-1] == '+': # проверкa 'кошка+', 'noun' or '+кошка', 'noun' or '+'
                res_words = []
                print('знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN')
                return res_words
            
            #поиск по POS и леммам. если плюс в середине чего-то напр bbb+aaa
            lemma, POS = word.split('+')
            
            # проверка на то что pos это pos
            if POS not in list_of_pos:
                res_words = []
                print('знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN')
                return res_words
        
            # проверка на то что lemma это лемма # кошкой+NOUN
            # лемматизируем наташей
            lem = get_lem(word)
                
            line = "Corpus.'3-gram@_lem' = " + "'" + lem + "'" + " AND Corpus.'3-gram@_pos' = " + "'" + POS + "'"
            
        else:
            lem = get_lem(word)
            #поиск по леммам
            line = "Corpus.'3-gram@_lem' = " + "'" + lem + "'"
        
        res_words.append(line)

    return res_words

In [8]:
# lemmatization
def get_lem(word): # на вход одно слово
    doc = Doc(word)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)   
    doc.tokens[0].lemmatize(morph_vocab) 
    # если сегментер разделит по дефису то возьмется лемма 1й половины
    lem = doc.tokens[0].lemma
    
    return lem

примеры подзапросов

In [29]:
# норм запрос
generate_request('кошек "нет" NOUN')

["Corpus.'3-gram@_lem' = 'кошка'",
 "Corpus.'3-gram@_orig' = 'нет'",
 "Corpus.'3-gram@_pos' = 'NOUN'"]

In [30]:
# не норм запрос
generate_request('коше+к "нет" NOUN')

знак + соединяет лемму/словоформу и часть речи. напр.: ехать+NOUN


[]

In [31]:
# ок запрос
generate_request('кошек "не+т" NOUN')

["Corpus.'3-gram@_lem' = 'кошка'",
 "Corpus.'3-gram@_orig' = 'не+т'",
 "Corpus.'3-gram@_pos' = 'NOUN'"]

In [32]:
# незакрытые кавычки
generate_request('кошек "нет NOUN')

Попробуйте закрыть двойные кавычки


[]

### функция, в которую надо вводить запросы:

In [9]:
def search(input_s):
    input_s = re.sub('\s+', ' ', input_s).strip() # замена повт пробелов на один
    check = len(input_s.split(' '))
    if check > 3 or check < 1:
        return 'Запрос некорректен'
    
    words = generate_request(input_s)
    if words == []:
        return 'Запрос некорректен'
        
    if check == 1:
        request = one_word_requests(words)
        print_scentences(request)
    elif check == 2:
        request = two_words_requests(words)
        print_scentences(request)
    elif check == 3:
        request = three_words_requests(words)
        print_scentences(request)

достает предложения из базы и выводит их

In [10]:
def extract_sents_id(case, c): # на вход id sents
    for el in case:
        req = "SELECT sent_orig, title_link FROM Sentences WHERE Sentences.'sent_id' = " + str(el[0])
        sent_orig, title_link = c.execute(req).fetchone()
        print('SENT: ' + sent_orig + '\n' + 'TITLE+URL: ' + title_link)
        print()

достает набор номеров предложений, отправляет его в функцию extract_sents_id

In [11]:
def print_scentences(request):
    conn = sqlite3.connect('project_db.db')
    c = conn.cursor()
    case = c.execute(request).fetchall()
    if case == []:
        print('По вашему запросу было найдено НИЧЕГО')
    else:
        extract_sents_id(case, c)
    c.close()
    conn.close()

генерация полных запросов в базу с использованием полученного подзапроса

In [12]:
def one_word_requests(words): # на вход list, ключи индексы 0, 1 или 2
    one_w_req = words[0]
    requests = []
    for index in range(3):
        ending = re.sub('@', str(index), one_w_req)
        requests.append(ending)
    res = 'SELECT DISTINCT sent FROM Corpus WHERE ' + ' OR '.join(requests) 
    # distinct чтобы без повторов
    
    return res # возвращает 1 запрос

Пример получаемого запроса в базу

In [38]:
one_word_requests(generate_request('кошек'))

"SELECT DISTINCT sent FROM Corpus WHERE Corpus.'3-gram0_lem' = 'кошка' OR Corpus.'3-gram1_lem' = 'кошка' OR Corpus.'3-gram2_lem' = 'кошка'"

генерация полных запросов в базу с использованием полученных подзапросов

In [13]:
def two_words_requests(words):
    requests = []
    indexes = [[0, 1], [1, 2]]
    for pair in indexes:
#         beginning = 'SELECT DISTINCT sent FROM Corpus WHERE '
        middle = re.sub('@', str(pair[0]), words[0])
        conj = ' AND '
        ending = re.sub('@', str(pair[1]), words[1])
        requests.append('(' + middle + conj + ending + ')')
    
    res = 'SELECT DISTINCT sent FROM Corpus WHERE ' + ' OR '.join(requests) 
    return res

Пример получаемого запроса в базу

In [40]:
two_words_requests(generate_request('кошек NOUN'))

"SELECT DISTINCT sent FROM Corpus WHERE (Corpus.'3-gram0_lem' = 'кошка' AND Corpus.'3-gram1_pos' = 'NOUN') OR (Corpus.'3-gram1_lem' = 'кошка' AND Corpus.'3-gram2_pos' = 'NOUN')"

генерация полных запросов в базу с использованием полученных подзапросов

In [14]:
def three_words_requests(words):
#     requests = []
#     indexes = [0, 1, 2]
    beginning = 'SELECT DISTINCT sent FROM Corpus WHERE '
    middle_first = re.sub('@', '0', words[0])
    middle_second = re.sub('@', '1', words[1])
    conj = ' AND '
    ending = re.sub('@', '2', words[2])
    res = beginning + middle_first + conj + middle_second + conj + ending
    
    return res

Пример получаемого запроса в базу

In [42]:
three_words_requests(generate_request('кошек NOUN поить+VERB'))

"SELECT DISTINCT sent FROM Corpus WHERE Corpus.'3-gram0_lem' = 'кошка' AND Corpus.'3-gram1_pos' = 'NOUN' AND Corpus.'3-gram2_lem' = 'поить' AND Corpus.'3-gram2_pos' = 'VERB'"

# Запуск!

### Примеры запросов
Приведите примеры запросов, которые можно задавать к вашему корпусу. 

(Ещё можно описать тематику, чтобы проверяющий мог придумать свои запросы)

Тематика корпуса: новостные статьи про Германию, сайт https://liveberlin.ru/news/

Набор тегов (из библиотека Natasha):

* AUX — вспомогательный глагол
* VERB — глагол
* ADV — наречие
* ADJ — прилагательное
* NUM — числительное
* DET — детерминатор
* NOUN — существительное
* PROPN — имя собственное
* PRON — местоимение
* CCONJ — сочинительный союз
* SCONJ — подчинительный союз
* INTJ — междометие
* PART — частица
* ADP — предлог
* X — иностранное слово
* SYM — символ

In [43]:
search('"как дела"')

Попробуйте закрыть двойные кавычки


'Запрос некорректен'

In [44]:
search('"Партия"')

SENT: Партия зеленых предложила ввести ограничение скорости 130 км/ч на немецких автобанах, с целью сократить количество выхлопных газов.
TITLE+URL: Ограничениям скорости на немецких автобанах не бывать!
https://liveberlin.ru/news/2019/11/04/ogranicheniyam-skorosti-na-nemetskih-avtobanah-ne-byvat/

SENT: По опросам, в Британии лидирует созданная в начале апреля этого года «Партия Brexit», занимающая крайне жесткую позицию в поддержку выхода страны из союза.
TITLE+URL: Великобритания примет участие в выборах в Европарламент несмотря на Brexit
https://liveberlin.ru/news/2019/05/09/velikobritaniya-primet-uchastie-v-vyborah-v-evroparlament-nesmotrya-na-brexit/

SENT: Партия этих рыбешек прибыла в начале марта из Гамбурга, куда они в свою очередь были доставлены с Атлантического побережья Франции, сообщает Berliner Morgenpost.
TITLE+URL: Экология: В берлинские реки выпустят два миллиона угрей
https://liveberlin.ru/news/2019/03/19/ekologiya-v-berlinskie-reki-vypustyat-dva-milliona-ugrej/



In [45]:
search('"Партия" VERB ADV')

По вашему запросу было найдено НИЧЕГО


In [20]:
search('Разъяснять')

SENT: Разъясняем тонкости.
TITLE+URL: У посетителей берлинских судов в прошлом году изъяли 22 тысячи единиц оружия и опасных предметов
https://liveberlin.ru/news/2019/02/20/u-posetitelej-berlinskih-sudov-v-proshlom-godu-izyali-22-000-edinits-oruzhiya-i-opasnyh-predmetov/



In [47]:
search('поиск')

SENT: Теперь, при помощи онлайн-ресурса, часть информации стала доступной каждому: для поиска достаточно ввести фамилию человека.
TITLE+URL: 13 миллионов документов о преступлениях национал-социалистов стали доступны онлайн
https://liveberlin.ru/news/2019/08/19/13-millionov-dokumentov-o-prestupleniyah-natsional-sotsialistov-stali-dostupny-onlajn/

SENT: Раздел поиска доступен на пяти языках, в том числе и на русском.
TITLE+URL: 13 миллионов документов о преступлениях национал-социалистов стали доступны онлайн
https://liveberlin.ru/news/2019/08/19/13-millionov-dokumentov-o-prestupleniyah-natsional-sotsialistov-stali-dostupny-onlajn/

SENT: Если соискатель не захочет или не сможет выполнять волонтерскую работу, он может от нее отказаться и дальше получать пособие или социальную помощь, а также продолжать поиски желаемого места.
TITLE+URL: Берлинские безработные, которые не могут найти работу больше года, смогут трудоустроиться как волонтеры
https://liveberlin.ru/news/2019/08/11/berlinski

In [48]:
search('"поиск"')

По вашему запросу было найдено НИЧЕГО


In [49]:
search('знать+NOUN')

По вашему запросу было найдено НИЧЕГО


In [17]:
search('NOUN VERB ADV')

SENT: При этом стоимость мобильной связи в Германии в международном сравнении оценивается выше среднего.
TITLE+URL: Связь 4G в Германии лучше, чем в Сенегале, но хуже, чем в Марокко
https://liveberlin.ru/news/2019/08/21/svyaz-4g-v-germanii-luchshe-chem-v-senegale-no-huzhe-chem-v-marokko/

SENT: Его фонд насчитывает более 30 миллионов документов с данными о периоде заключения, принудительных работах, а также о помощи союзников в послевоенное время.
TITLE+URL: 13 миллионов документов о преступлениях национал-социалистов стали доступны онлайн
https://liveberlin.ru/news/2019/08/19/13-millionov-dokumentov-o-prestupleniyah-natsional-sotsialistov-stali-dostupny-onlajn/

SENT: В Берлине 85% жилья сдается внаем.
TITLE+URL: Рост цен на аренду жилья в Берлине снижен до 2,5% в год
https://liveberlin.ru/news/2019/08/13/rost-tsen-na-arendu-zhilya-v-berline-snizhen-do-25-v-god/

SENT: С 1 августа 2019 года берлинские школьники могут бесплатно пользоваться городским общественным транспортом в зонах A 

In [51]:
search('ADJ дом')

SENT: Определяющей исторической причиной этого специалисты называют значительно пострадавший или полностью разрушенный жилой фонд в годы Второй мировой войны и последующее бурное строительство многоквартирных домов под аренду как в ГДР, так и в ФРГ.
TITLE+URL: Более половины немцев живут в съемном жилье
https://liveberlin.ru/news/2019/03/06/bolee-poloviny-nemtsev-zhivut-v-semnom-zhile/

SENT: Решение действует в ситуациях, когда для высокоскоростного подключения того или иного дома к интернету Deutsche Telekom применяет технологию Super-Vectoring (возможная скорость до 250 Mbit/s), а альтернативный провайдер предоставляет услуги оптоволоконной сети по стандарту G.fast (порядка 1 GBit/s).
TITLE+URL: Deutsche Telekom сможет ограничивать работу других интернет-провайдеров и даже полностью отключать их
https://liveberlin.ru/news/2019/01/23/deutsche-telekom-smozhet-ogranichivat-rabotu-drugih-internet-provajderov-i-dazhe-polnostyu-otklyuchat-ih/



In [53]:
search('X VERB')

SENT: В сентябре 2019 года уполномоченная по защите данных и свободе информации Берлина Майя Смольчик (Maja Smoltczyk, Berliner Beauftragte für Datenschutz und Informationsfreiheit) оштрафовала сервис доставки еды Delivery Hero на 200 000 евро.
TITLE+URL: Регламент о защите данных (DSGVO/GDPR) заработал в Германии в полную силу. И это очень жестко
https://liveberlin.ru/news/2019/10/30/reglament-o-zaschite-dannyh-dsgvo-gdpr-zarabotal-v-germanii-v-polnuyu-silu-i-eto-ochen-zhestko/

SENT: Delivery Hero пытались списать эти случаи на технические сбои или ошибки своих сотрудников, но берлинская уполномоченная по защите данных оказалась сурова: она заявила, что эти нарушения вызваны «фундаментальными, структурными организационными проблемами».
TITLE+URL: Регламент о защите данных (DSGVO/GDPR) заработал в Германии в полную силу. И это очень жестко
https://liveberlin.ru/news/2019/10/30/reglament-o-zaschite-dannyh-dsgvo-gdpr-zarabotal-v-germanii-v-polnuyu-silu-i-eto-ochen-zhestko/

SENT: * * * 